In [1]:
spark

In [2]:
# !hdfs dfs -stat /project/awg/cms/rucio/2023-07-31/

In [2]:
!hdfs dfs -ls /project/awg/cms/rucio/2023-07-24/rules_history #02:54:14

23/08/01 17:05:11 WARN ipc.Client: Exception encountered while connecting to the server 
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
	at org.apache.hadoop.security.SaslRpcClient.saslConnect(SaslRpcClient.java:376)
	at org.apache.hadoop.ipc.Client$Connection.setupSaslConnection(Client.java:622)
	at org.apache.hadoop.ipc.Client$Connection.access$2300(Client.java:413)
	at org.apache.hadoop.ipc.Client$Connection$2.run(Client.java:822)
	at org.apache.hadoop.ipc.Client$Connection$2.run(Client.java:818)
	at java.base/java.security.AccessController.doPrivileged(Native Method)
	at java.base/javax.security.auth.Subject.doAs(Subject.java:423)
	at org.apache.hadoop.security.UserGroupInformation.doAs(UserGroupInformation.java:1845)
	at org.apache.hadoop.ipc.Client$Connection.setupIOstreams(Client.java:818)
	at org.apache.hadoop.ipc.Client$Connection.access$3800(Client.

In [3]:
import pickle
from datetime import datetime, timedelta

import click
import os
import pandas as pd
import pprint
import time
from dateutil.relativedelta import relativedelta
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, collect_list, concat_ws, greatest, lit, lower, when,
    avg as _avg,
    count as _count,
    hex as _hex,
    max as _max,
    min as _min,
    round as _round,
    sum as _sum,
)

from pyspark.sql.types import (
    LongType,
)

#from CMSSpark.src.python.CMSSpark import schemas as cms_schemas

In [4]:
#from CMSSpark import schemas as cms_schemas

## load dataset

In [16]:
# end_date = str(datetime.now())[:10]
# start_date = str(datetime.now()-timedelta(days=1))[:10]

end_date = '2023-07-31'
start_date = '2023-07-01'

TOYEAR = end_date[:4]

wa_date = end_date
HDFS_RUCIO_DATASET_LOCKS = f'/project/awg/cms/rucio/{wa_date}/dataset_locks/part*.avro'
HDFS_RUCIO_LOCKS =         f'/project/awg/cms/rucio/{wa_date}/locks'
HDFS_RUCIO_RSES =          f'/project/awg/cms/rucio/{wa_date}/rses/part*.avro'
HDFS_RUCIO_RULES =         f'/project/awg/cms/rucio/{wa_date}/rules'
HDFS_RUCIO_RULES_HISTORY = f'/project/awg/cms/rucio/{wa_date}/rules_history'
HDFS_RUCIO_REPLICAS =      f'/project/awg/cms/rucio/{wa_date}/replicas'

In [6]:
# rucio_dataset_locks = spark.read.format('avro').load(HDFS_RUCIO_DATASET_LOCKS)\
#     .withColumn('BYTES', col('BYTES').cast(LongType()))\
#     .withColumn('RULE_ID', lower(_hex(col('RULE_ID'))))\
#     .withColumn('RSE_ID', lower(_hex(col('RSE_ID'))))
# rucio_dataset_locks.createOrReplaceTempView("dataset_locks")

In [7]:
# rucio_rses = spark.read.format('avro').load(HDFS_RUCIO_RSES)\
#     .withColumn('ID', lower(_hex(col('ID'))))
# rucio_rses.createOrReplaceTempView("rses")


In [8]:
# rucio_rules = spark.read.format('avro').load(HDFS_RUCIO_RULES)\
#     .withColumn('ID', lower(_hex(col('ID'))))
# rucio_rules.createOrReplaceTempView("rules")
# #spark.sql("SELECT * FROM rules").count()

In [9]:
# rucio_locks = spark.read.format('avro').load(HDFS_RUCIO_LOCKS)\
#     .withColumn('BYTES', col('BYTES').cast(LongType()))\
#     .withColumn('RULE_ID', lower(_hex(col('RULE_ID'))))\
#     .withColumn('RSE_ID', lower(_hex(col('RSE_ID'))))
# rucio_locks.createOrReplaceTempView("locks")

In [13]:
rucio_rules_history = spark.read.format('avro').load(HDFS_RUCIO_RULES_HISTORY)\
    .withColumn('ID', lower(_hex(col('ID'))))
    #.persist(StorageLevel.DISK_ONLY)
rucio_rules_history = rucio_rules_history.select("ID", "NAME", "STATE", "EXPIRES_AT", "UPDATED_AT", "CREATED_AT", "ACCOUNT")
rucio_rules_history.createOrReplaceTempView("rules_history")
#spark.sql("SELECT * FROM rules_history").count()

In [14]:
# rucio_replicas = spark.read.format('avro').load(HDFS_RUCIO_REPLICAS)\
#     .withColumn('RSE_ID', lower(_hex(col('RSE_ID'))))
# rucio_replicas.createOrReplaceTempView("replicas")
# #spark.sql("SELECT * FROM replicas").count()

## Query

# how long does it take ?

In [15]:
# # NOTE: days is ceiling

# spark.sql("""
# WITH filter_t AS (
# SELECT ID, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT
# FROM rules_history 
# WHERE 1=1
# AND ACCOUNT = "crab_tape_recall"
# --- we look at the rule created this year (2023)
# AND CREATED_AT >= unix_timestamp("2023-01-01 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 
# ),
# rn_t AS (
# SELECT ID, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT,
# row_number() over(partition by ID order by UPDATED_AT desc) as rn --- to get only latest state for each id
# FROM filter_t
# ),
# calc_days_t AS (
# SELECT ID, NAME, STATE, 
# from_unixtime(EXPIRES_AT/1000, 'yyyy-MM-dd HH:mm:ss') AS EXPIRES_AT, 
# from_unixtime(UPDATED_AT/1000, 'yyyy-MM-dd HH:mm:ss') AS UPDATED_AT, 
# from_unixtime(CREATED_AT/1000, 'yyyy-MM-dd HH:mm:ss') AS CREATED_AT,
# --- if state is O we calculate from update_at when state change (assumed that there is only single row for O state)
# --- but if state is not O, we calculate from expired time, it usually 14 days but it is possible that rules somehow got extend
# --- other wise days = 0 for filter the rules that not expire 
#    CASE 
#       WHEN STATE = 'O' THEN ceil((UPDATED_AT-CREATED_AT)/86400000)  
#       WHEN STATE != 'O' AND EXPIRES_AT < unix_timestamp("2023-05-22 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 THEN ceil((EXPIRES_AT-CREATED_AT)/86400000)
#       ELSE 0
#    END AS DAYS
# FROM rn_t
# WHERE rn = 1
# )
# SELECT * 
# FROM calc_days_t
# ---AND STATE == 'O'
# """
# ).show(50,truncate=False)

In [19]:
## query use to produce data to elasticsearch

query = f"""\
WITH filter_t AS (
SELECT ID, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT
FROM rules_history 
WHERE 1=1
AND ACCOUNT = "crab_tape_recall"
AND CREATED_AT >= unix_timestamp("{TOYEAR}-01-01 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000
),
rn_t AS (
SELECT ID, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT,
row_number() over(partition by ID order by UPDATED_AT desc) as rn
FROM filter_t
),
calc_days_t AS (
SELECT ID, NAME, STATE, EXPIRES_AT, UPDATED_AT, CREATED_AT,
   CASE 
      WHEN STATE = 'O' THEN ceil((UPDATED_AT-CREATED_AT)/86400000)  
      WHEN STATE != 'O' AND EXPIRES_AT < unix_timestamp("{wa_date} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 THEN ceil((EXPIRES_AT-CREATED_AT)/86400000)
      ELSE 0
   END AS DAYS
FROM rn_t
WHERE rn = 1
)
SELECT * 
FROM calc_days_t
WHERE 1=1
AND EXPIRES_AT >= unix_timestamp("{start_date} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 
AND EXPIRES_AT < unix_timestamp("{end_date} 00:00:00", "yyyy-MM-dd HH:mm:ss")*1000 
"""

tmpdf = spark.sql(query)
# str(datetime.now()-timedelta(days=1))[:10]

In [20]:
tmpdf.show(50)

+--------------------+--------------------+-----+-------------+-------------+-------------+----+
|                  ID|                NAME|STATE|   EXPIRES_AT|   UPDATED_AT|   CREATED_AT|DAYS|
+--------------------+--------------------+-----+-------------+-------------+-------------+----+
|16e7eeb0a6c447839...|/DYJetsToLL_LHEFi...|    O|1689496342000|1689130290000|1686566168000|  30|
|27aea75d1d364b219...|/WJetsToLNu_HT-20...|    O|1689568449000|1689086563000|1686931142000|  25|
|3f2d7fcff69d49079...|/ParkingBPH1/Run2...|    R|1689522386000|1687621610000|1687621586000|  22|
|67d9f565492b4dec9...|/DYJetsToLL_M-10t...|    R|1689519133000|1687618376000|1687618333000|  22|
|c2cbad3267e84ba18...|/TapeRecall/23061...|    O|1689554004000|1689117261000|1686940766000|  26|
|d23ee08f6aac4d5db...|/QCD_HT300to500_T...|    O|1689525153000|1689048723000|1686900417000|  25|
|ddfdfed2239940298...|/W2JetsToLNu_Tune...|    R|1689517301000|1687616515000|1687616501000|  22|
|dee8dbd0a82b48b59...|/TapeRec

In [1]:
tmpdf.printSchema()

In [47]:
docs = tmpdf.toPandas().to_dict('records')

In [48]:
len(docs)

8260

In [49]:
docs[:5]

[{'ID': '00049b4efb3e4dd091dbfed2012069df',
  'NAME': '/TapeRecall/221110_230609.dshmygol_crab_Bfinder_2018_MC_Bc_in_JpsiPI_v0_1/USER',
  'STATE': 'O',
  'EXPIRES_AT': 1669331191000,
  'UPDATED_AT': 1668133187000,
  'CREATED_AT': 1668121591000,
  'DAYS': 1},
 {'ID': '0007a18199834a2ca720f088d96a3c9c',
  'NAME': '/TapeRecall/220427_065307.youying_crab_DiphoVtxUL2016_DoubleMuon_Run2016B-21Feb2020_ver2_UL2016_HIPM-v1/USER',
  'STATE': 'O',
  'EXPIRES_AT': 1652252233000,
  'UPDATED_AT': 1651048717000,
  'CREATED_AT': 1651042633000,
  'DAYS': 1},
 {'ID': '00d4ba364b89477e888e8797a33092d2',
  'NAME': '/TapeRecall/210810_035101.jingqing_crab_BPHSkimOfficialChib06900-2016-v5/USER',
  'STATE': 'O',
  'EXPIRES_AT': 1629777107000,
  'UPDATED_AT': 1628847357000,
  'CREATED_AT': 1628567507000,
  'DAYS': 4},
 {'ID': '00fb74e1bafc40aba0736216b798a80c',
  'NAME': '/TapeRecall/230220_091052.shiyi_crab_RUN3_2022Dv2mass3_SKIM_E_newV2/USER',
  'STATE': 'R',
  'EXPIRES_AT': 1678093905000,
  'UPDATED_AT': 1

In [52]:
import osearch

In [53]:
def get_index_schema():
    return {
        "settings": {"index": {"number_of_shards": "1", "number_of_replicas": "1"}},
        "mappings": {
            "properties": {
                "timestamp": {"format": "epoch_second", "type": "date"},
                "ID": {"ignore_above": 1024, "type": "keyword"},
                "NAME": {"ignore_above": 2048, "type": "keyword"},
                "STATE": {"ignore_above": 1024, "type": "keyword"},
                "EXPIRES_AT": {"format": "epoch_millis", "type": "date"},
                "UPDATED_AT": {"format": "epoch_millis", "type": "date"},
                "CREATED_AT": {"format": "epoch_millis", "type": "date"},
                "DAYS": {"type": "long"},
            }
        }
    }

In [54]:
_index_template = 'crab-tape-recall-daily-ekong'
client = osearch.get_es_client("es-cms1.cern.ch/es", 'secret_opensearch.txt', get_index_schema())
# index_mod="": 'test-foo', index_mod="Y": 'test-foo-YYYY', index_mod="M": 'test-foo-YYYY-MM', index_mod="D": 'test-foo-YYYY-MM-DD',
idx = client.get_or_create_index(timestamp=time.time(), index_template=_index_template, index_mod="M")
client.send(idx, docs, metadata=None, batch_size=10000, drop_nulls=False)

/eos/user/e/eatthaph/.local/lib/python3.9/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://es-cms1.cern.ch:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


0